# Airbnb Reviews Pipeline - MiniLM + SST-2

**Goal:** Extract negative, issue-tagged sentences from Airbnb reviews and produce actionable aggregations (property + host).

## Pipeline stages
1) Load + filter Italy
2) Split reviews blob → review chunks (review-level)
3) Split review chunks → sentences (English-only, min_words, smart dedup)
4) Top-K per property prefilter (reduce volume before embeddings)
5) Similarity (MiniLM) on candidate unique sentences and Apply similarity thresholds → issue candidates
7) Sentiment (SST-2) on issue-candidates (sharded + resume)
8) Join (issue × negative) → final issues (unique + occurrences)
9) Aggregations + evidence tables
10) LLM recommendations API on selected properties.

## Locked decisions
- Sentence filters: English-only (ASCII), `min_words=5`, smart normalization + dedup by `sentence_norm`
- Similarity: MiniLM computed once (no second pass)
- Similarity thresholds: `ISSUE_SIM_THRESHOLD=0.4`, `TOP2_GAP_MAX=0.04` (Top2 only if very close)
- Sentiment: SST-2, keep if `neg_prob >= 0.40`
- Outputs: final issues tables + LLM recommendations 


In [0]:
storage_account = "replace_by_storage_account"  
container = "replace_by_container"

In [0]:
sas_token="replace_with_your_sas_token"
sas_token = sas_token.lstrip('?')
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)

In [0]:
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet"

airbnb = spark.read.parquet(path)
display(airbnb.limit(5))

In [0]:
from pyspark.sql import functions as F

df = airbnb.withColumn("country_from_location",
                       F.trim(F.element_at(F.split(F.col("location"), ","), -1)))

df_italy = df.filter(F.upper(F.col("country_from_location")) == "ITALY")

print("[Italy] rows:", df_italy.count())
print("[Italy] unique properties:", df_italy.select("property_id").distinct().count())

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType, StructField, StringType, FloatType, BooleanType
)
import pandas as pd
import numpy as np
from datetime import datetime

OUTPUT_BASE = "dbfs:/FileStore/airbnb"
RUN_ID = "ITALY_1"
print(f"[INFO] RUN_ID={RUN_ID}")

def save_table_and_path(df, table_name: str, mode: str = "overwrite"):
    # keep as metastore table (for spark.table usage)
    df.write.mode(mode).format("delta").saveAsTable(table_name)

    # save a Delta copy to DBFS path
    out_path = f"{OUTPUT_BASE}/{table_name}"
    df.write.mode(mode).format("delta").save(out_path)
    print(f"[OK] Saved table={table_name} | path={out_path}")

# Column names
PROPERTY_COL   = "property_id"
SELLER_INFO_COL = "seller_info"
SELLER_ID_COL  = "seller_id"
REVIEWS_COL    = "reviews"  

REVIEW_ID_COL_CANDIDATES = ["review_id", "id", "reviewId"]

# Thresholds (locked) 
MIN_WORDS = 5

ISSUE_SIM_THRESHOLD = 0.4
TOP_GAP_MIN = 0.05
TOP2_GAP_MAX = 0.04

NEG_PROB_THRESHOLD = 0.40

# Models
SIM_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
SENT_MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

# Output tables
T_REVIEWS_BASE          = f"airbnb_reviews_base_{RUN_ID}"

T_SENTENCES_OCCUR       = f"airbnb_sentences_occurrences_{RUN_ID}"
T_SENTENCES_UNIQUE      = f"airbnb_sentences_unique_{RUN_ID}"

T_SIM_UNIQUE            = f"airbnb_similarity_unique_{RUN_ID}"

T_SENTIMENT_UNIQUE      = f"airbnb_sentiment_unique_{RUN_ID}"
T_NEG_UNIQUE            = f"airbnb_negative_unique_{RUN_ID}"
T_NEG_OCCUR             = f"airbnb_negative_occurrences_{RUN_ID}"

T_ISSUES_UNIQUE          = f"airbnb_issues_unique_{RUN_ID}"
T_ISSUES_OCCUR           = f"airbnb_issues_occurrences_{RUN_ID}"

T_PROP_ISSUE_AGG         = f"airbnb_property_issue_agg_{RUN_ID}"
T_HOST_ISSUE_AGG         = f"airbnb_host_issue_agg_{RUN_ID}"

T_PROP_ISSUE_EVID        = f"airbnb_property_issue_evidence_{RUN_ID}"
T_HOST_ISSUE_EVID        = f"airbnb_host_issue_evidence_{RUN_ID}"

T_ISSUES_CANDIDATES_UNIQUE = f"airbnb_issues_candidates_unique_{RUN_ID}"
T_ISSUES_CANDIDATES_OCCUR  = f"airbnb_issues_candidates_occurrences_{RUN_ID}"

T_SENTENCES_CANDIDATES_OCCUR  = f"airbnb_sentences_candidates_occurrences_{RUN_ID}"
T_SENTENCES_CANDIDATES_UNIQUE = f"airbnb_sentences_candidates_unique_{RUN_ID}"

T_SENT_PROGRESS = f"airbnb_sent_progress_{RUN_ID}"
T_SENT_INPUT    = f"airbnb_sent_input_{RUN_ID}"

print("[INFO] Output tables:")
for t in [
    T_REVIEWS_BASE, T_SENTENCES_OCCUR, T_SENTENCES_UNIQUE, T_SIM_UNIQUE,
    T_SENTIMENT_UNIQUE, T_NEG_UNIQUE, T_NEG_OCCUR, T_ISSUES_UNIQUE, T_ISSUES_OCCUR,
    T_PROP_ISSUE_AGG, T_HOST_ISSUE_AGG, T_PROP_ISSUE_EVID, T_HOST_ISSUE_EVID
]:
    print(" -", t)

## Issues taxonomy (final approved)

In [0]:
ISSUE_TEXTS = {
    "wifi": (
        "internet connection quality, wifi speed, weak or unstable signal, "
        "slow internet, connection drops, unreliable wifi for work or streaming"
    ),
    "tv_and_entertainment": (
        "television, TV functionality, smart TV, streaming services, Netflix, "
        "channels, media system, remote control issues, entertainment setup"
    ),
    "workspace": (
        "workspace suitability, desk and chair, working remotely, laptop setup, "
        "comfortable space for work or study, home office conditions"
    ),
    "climate_control": (
        "air conditioning, heating system, temperature control, cooling or warmth, "
        "poor ventilation, too hot or too cold inside the property"
    ),
    "furniture_and_comfort": (
        "bed comfort, mattress quality, sofa or couch comfort, broken furniture, "
        "uncomfortable seating, sleeping quality, worn or damaged furniture"
    ),
    "cleanliness": (
        "cleanliness and hygiene, dirty or dusty apartment, unpleasant smells, "
        "stains, poor cleaning standards, lack of sanitation"
    ),
    "linens": (
        "bed linens, sheets, towels, blankets, linen cleanliness or freshness, "
        "old, dirty, or insufficient towels or bedding"
    ),
    "bathroom_and_water": (
        "bathroom condition, shower quality, toilet functionality, water pressure, "
        "hot water availability, plumbing problems, mold or mildew in bathroom"
    ),
    "noise": (
        "noise levels, loud neighbors, street noise, construction sounds, "
        "poor sound insulation, difficult to sleep due to noise"
    ),
    "space_and_privacy": (
        "apartment size, cramped or small space, layout and room arrangement, "
        "lack of privacy, shared spaces, inconvenient floor plan or room separation"
    ),
    "kitchen_and_appliances": (
        "kitchen facilities, cooking equipment, stove, oven, refrigerator, microwave, "
        "coffee machine, kettle, broken or missing kitchen appliances"
    ),
    "rules_and_access": (
        "check-in and check-out process, access to the property, keys or door codes, "
        "house rules, unclear instructions, restrictions, confusing entry or exit"
    ),
    "parking": (
        "parking availability, parking convenience, garage access, street parking, "
        "difficulty finding parking near the property"
    ),
    "accessibility": (
        "accessibility issues, elevator or lift availability, stairs only access, "
        "difficulty reaching the apartment, suitability for limited mobility"
    ),
    "location": (
        "location convenience, neighborhood quality, distance to attractions, "
        "public transportation access, safety or comfort of the surrounding area"
    ),
    "view": (
        "view from the property, balcony or window scenery, sea or city view, "
        "blocked or disappointing view compared to listing description"
    ),
    "safety": (
        "safety and security concerns, feeling unsafe, poor locks, building security, "
        "unsafe neighborhood or entrance"
    ),
    "property_condition": (
        "maintenance issues, broken items, poor condition, lack of repairs, "
        "lighting problems, insufficient storage space, overall property upkeep"
    ),
    "host_communication": (
        "communication with the host, slow or no response, lack of helpfulness, "
        "difficulty resolving problems, unresponsive or unclear communication"
    ),
    "pets": (
        "pet friendliness, pet restrictions, pet fees, issues with pets, "
        "unexpected rules regarding animals"
    ),
    "price_value": (
        "value for money, overpriced stay, price not matching quality, "
        "expectations versus cost, poor value compared to similar listings"
    ),
}
ISSUES = list(ISSUE_TEXTS.keys())
print("[INFO] #ISSUES =", len(ISSUES))

## Step 0 - Base table: extract seller_id + drop nulls + (property_id, reviews) block-dedup

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType

df_base = df_italy

# Validate minimal inputs exist 
missing_inputs = [c for c in [PROPERTY_COL, REVIEWS_COL, SELLER_INFO_COL]if c not in df_base.columns]
if missing_inputs:
    raise Exception(f"[ERROR] Missing required columns in airbnb: {missing_inputs}\nFound: {df_base.columns}")

# Extract seller_id from seller_info JSON 
seller_schema = StructType([ StructField("seller_id", StringType(), True), StructField("url", StringType(), True), StructField("name", StringType(), True), ])
df_base = df_base.withColumn( "seller_parsed", F.from_json(F.col(SELLER_INFO_COL).cast("string"), seller_schema) ).withColumn( SELLER_ID_COL, F.col("seller_parsed.seller_id") ).drop("seller_parsed")
print("[OK] seller_id extracted from seller_info")

# Drop rows with missing critical keys/text
df_base = df_base.filter( F.col(PROPERTY_COL).isNotNull() & (F.length(F.col(PROPERTY_COL).cast("string")) > 0) & F.col(SELLER_ID_COL).isNotNull() & (F.length(F.col(SELLER_ID_COL).cast("string")) > 0) & F.col(REVIEWS_COL).isNotNull() & (F.length(F.trim(F.col(REVIEWS_COL).cast("string"))) > 0) )

# Keep only required cols downstream
df_base = df_base.select(PROPERTY_COL, SELLER_ID_COL, REVIEWS_COL)

# Basic quality checks (after filtering)
total_rows = df_base.count()
print("[INFO] rows (after dropping null keys/text):", total_rows)
print("[INFO] unique properties:",df_base.select(PROPERTY_COL).distinct().count())
print("[INFO] unique sellers:", df_base.select(SELLER_ID_COL).distinct().count())
print("[DEBUG] sample rows:")
display(df_base.limit(10))

# Save base table for the pipeline  
save_table_and_path(df_base, T_REVIEWS_BASE)
print("[INFO] base rows:", df_base.count())

In [0]:
# Remove duplicated reviews inside the same property before sentence splitting.
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
import re
import pandas as pd

# Output table for this step
T_REVIEW_CHUNKS_BASE = f"airbnb_review_chunks_base_{RUN_ID}"
print("[INFO] Output:", T_REVIEW_CHUNKS_BASE)

df0 = spark.table(T_REVIEWS_BASE)

# Basic checks
print("[INFO] T_REVIEWS_BASE rows:", df0.count())
display(df0.select(PROPERTY_COL, SELLER_ID_COL, REVIEWS_COL).limit(5))

# Schema for occurrences (before dedup)
chunks_schema = StructType([
    StructField(PROPERTY_COL, StringType(), True),
    StructField(SELLER_ID_COL, StringType(), True),
    StructField("review_chunk", StringType(), True),
])

def split_reviews_to_chunks(pdf_iter):
    """
    Split the 'reviews' blob into individual reviews FIRST.
    Primary rule: split on the delimiter  ","  (typical for scraped list-of-quotes reviews).
    Fallback: paragraph/newline/bullets split.
    """
    # Detect "quoted reviews list" delimiter:  ","  (allow whitespace/newlines between)
    # Example: ...hotspots.","A perfect stay...<br/>Straight down...
    review_delim_re = re.compile(r'"\s*,\s*"')

    # Fallback paragraph-ish split (kept from your original logic, slightly simplified)
    fallback_split_re = re.compile(r"(?:\n\s*\n)+|(?:\n[\-\*\u2022]\s+)|(?:\n{1,})")

    # HTML <br/> -> newline
    br_re = re.compile(r"<br\s*/?>", flags=re.IGNORECASE)

    # Strip outer wrappers like ["...","..."] or leading/trailing quotes
    strip_wrappers_re = re.compile(r'^\s*\[\s*"|"\s*\]\s*$')

    for pdf in pdf_iter:
        props = pdf[PROPERTY_COL].astype(str).tolist()
        sellers = pdf[SELLER_ID_COL].astype(str).tolist()
        blobs = pdf[REVIEWS_COL].fillna("").astype(str).tolist()

        out = []
        for i, blob in enumerate(blobs):
            t = (blob or "").strip()
            if not t:
                continue

            # normalize HTML breaks
            t = br_re.sub("\n", t)

            # If it looks like a quoted-list blob, split by "," between quotes
            parts = []
            if '","' in t or review_delim_re.search(t):
                # remove outer wrappers if exist
                t2 = strip_wrappers_re.sub("", t)
                # split by delimiter
                raw_parts = review_delim_re.split(t2)

                # clean each review: strip stray quotes/commas/spaces
                for p in raw_parts:
                    p = (p or "").strip()
                    p = p.strip(' "')
                    p = p.strip()
                    if p:
                        parts.append(p)

            # Fallback: if delimiter logic didn’t produce anything
            if not parts:
                parts = [p.strip() for p in fallback_split_re.split(t) if p and p.strip()]
                if not parts:
                    parts = [t]

            for ch in parts:
                ch = re.sub(r"\s+", " ", ch).strip()
                if len(ch) < 20:
                    continue
                out.append({
                    PROPERTY_COL: props[i],
                    SELLER_ID_COL: sellers[i],
                    "review_chunk": ch
                })

        yield pd.DataFrame(out, columns=[PROPERTY_COL, SELLER_ID_COL, "review_chunk"])

# 1) Split to chunk occurrences
df_chunks_occ = (
    df0.select(PROPERTY_COL, SELLER_ID_COL, REVIEWS_COL)
       .repartition(400, SELLER_ID_COL)
       .mapInPandas(split_reviews_to_chunks, schema=chunks_schema)
)

rows_chunks_occ = df_chunks_occ.count()
print(f"[INFO] review_chunk occurrences (before dedup): {rows_chunks_occ:,}")
display(df_chunks_occ.select(F.length("review_chunk").alias("len")).summary("min","25%","50%","mean","75%","max"))

# 2) Normalize + dedup per (property_id, review_chunk_norm)
df_chunks_occ = df_chunks_occ.withColumn("review_chunk", F.trim(F.col("review_chunk")))
df_chunks_occ = df_chunks_occ.withColumn("review_chunk", F.regexp_replace(F.col("review_chunk"), r"\s+", " "))

df_chunks_occ = df_chunks_occ.withColumn("review_chunk_norm", F.lower(F.col("review_chunk")))
df_chunks_occ = df_chunks_occ.withColumn("review_chunk_norm", F.regexp_replace(F.col("review_chunk_norm"), r"\s+", " "))
df_chunks_occ = df_chunks_occ.withColumn("review_chunk_norm", F.trim(F.col("review_chunk_norm")))

# Apply dedup
df_chunks_unique = df_chunks_occ.dropDuplicates([PROPERTY_COL, "review_chunk_norm"])

rows_chunks_unique = df_chunks_unique.count()
removed = rows_chunks_occ - rows_chunks_unique
print(f"[OK] Chunk dedup done: unique={rows_chunks_unique:,} | removed={removed:,} ({(removed/rows_chunks_occ):.2%} saved)")

# QA: show properties with most removed chunks
prop_before = df_chunks_occ.groupBy(PROPERTY_COL).count().withColumnRenamed("count", "chunks_before")
prop_after  = df_chunks_unique.groupBy(PROPERTY_COL).count().withColumnRenamed("count", "chunks_after")

prop_delta = (
    prop_before.join(prop_after, on=PROPERTY_COL, how="inner")
               .withColumn("removed_chunks", F.col("chunks_before") - F.col("chunks_after"))
               .orderBy(F.desc("removed_chunks"))
)

print("[DEBUG] Properties with most duplicated chunks removed:")
display(prop_delta.limit(30))

# 3) Add stable ids for downstream joins / traceability
df_chunks_unique = df_chunks_unique.withColumn("review_chunk_id", F.sha2(F.col("review_chunk_norm"), 256))

# Keep only needed cols
df_chunks_unique = df_chunks_unique.select(
    PROPERTY_COL, SELLER_ID_COL,
    "review_chunk_id",
    "review_chunk",
    "review_chunk_norm"
)

# 4) Save
save_table_and_path(df_chunks_unique, T_REVIEW_CHUNKS_BASE)
print("[INFO] review_chunks unique:", df_chunks_unique.count())


In [0]:
from pyspark.sql import functions as F

print("[QA] Example raw reviews blob (first row):")
display(df0.select(REVIEWS_COL).limit(1))

print("[QA] Sample produced review_chunks:")
display(df_chunks_occ.select(PROPERTY_COL, "review_chunk", F.length("review_chunk").alias("len")).limit(10))

print("[QA] How many chunks per property (top 20):")
display(
    df_chunks_occ.groupBy(PROPERTY_COL)
                 .agg(F.count("*").alias("n_chunks"))
                 .orderBy(F.desc("n_chunks"))
                 .limit(20)
)


## Step 1 - Sentence splitting + English-only + min_words + smart normalization (for dedup)
**Outputs:**
- `T_SENTENCES_OCCUR`: all sentence occurrences (keeps property/host linkage)
- `T_SENTENCES_UNIQUE`: unique sentences by `sentence_norm` for model inference

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

TEXT_COL = "review_chunk"

occur_schema = StructType([
    StructField(PROPERTY_COL, StringType(), True),
    StructField(SELLER_ID_COL, StringType(), True),
    StructField("review_chunk_id", StringType(), True),
    StructField("sentence", StringType(), True),
])

def split_to_sentences(pdf_iter):
    import re
    splitter = re.compile(r"(?<=[\.\!\?])\s+|\n+")

    for pdf in pdf_iter:
        out_rows = []
        prop   = pdf[PROPERTY_COL].astype(str).tolist()
        seller = pdf[SELLER_ID_COL].astype(str).tolist()
        cid    = pdf["review_chunk_id"].astype(str).tolist()
        texts  = pdf[TEXT_COL].fillna("").astype(str).tolist()

        for i, text in enumerate(texts):
            t = (text or "").strip()
            if not t:
                continue

            parts = [p.strip() for p in splitter.split(t) if p and p.strip()]
            for s in parts:
                out_rows.append({
                    PROPERTY_COL: prop[i],
                    SELLER_ID_COL: seller[i],
                    "review_chunk_id": cid[i],
                    "sentence": s
                })

        yield pd.DataFrame(out_rows, columns=[PROPERTY_COL, SELLER_ID_COL, "review_chunk_id", "sentence"])


df_in = spark.table(T_REVIEW_CHUNKS_BASE)

df_occ = (
    df_in
    .select(PROPERTY_COL, SELLER_ID_COL, "review_chunk_id", F.col(TEXT_COL).alias(TEXT_COL))
    .repartition(400, SELLER_ID_COL)
    .mapInPandas(split_to_sentences, schema=occur_schema)
)

# clean
df_occ = df_occ.withColumn("sentence", F.trim(F.col("sentence")))
df_occ = df_occ.withColumn("sentence", F.regexp_replace(F.col("sentence"), r"\s+", " "))

# min_words
df_occ = df_occ.withColumn("n_words", F.size(F.split(F.col("sentence"), r"\s+")))
df_occ = df_occ.filter(F.col("n_words") >= F.lit(int(MIN_WORDS)))

# english-only heuristic (strict ASCII + some latin letters)
df_occ = df_occ.withColumn("non_ascii", F.length(F.regexp_replace(F.col("sentence"), r"[\x00-\x7F]", "")))
df_occ = df_occ.withColumn("latin_letters", F.length(F.regexp_replace(F.col("sentence"), r"[^A-Za-z]", "")))
df_occ = df_occ.filter((F.col("non_ascii") <= F.lit(0)) & (F.col("latin_letters") >= F.lit(3)))

# norm for smart dedup
df_occ = df_occ.withColumn("sentence_norm", F.lower(F.col("sentence")))
df_occ = df_occ.withColumn("sentence_norm", F.regexp_replace(F.col("sentence_norm"), r"\s+", " "))
df_occ = df_occ.withColumn("sentence_norm", F.regexp_replace(F.col("sentence_norm"), r"^[\.\,\!\?\:\;\-]+", ""))
df_occ = df_occ.withColumn("sentence_norm", F.regexp_replace(F.col("sentence_norm"), r"[\.\,\!\?\:\;\-]+$", ""))
df_occ = df_occ.withColumn("sentence_norm", F.trim(F.col("sentence_norm")))

# stable id
df_occ = df_occ.withColumn("sentence_id", F.sha2(F.col("sentence_norm"), 256))

save_table_and_path(
    df_occ.select(PROPERTY_COL, SELLER_ID_COL, "review_chunk_id",
                  "sentence_id", "sentence", "sentence_norm", "n_words"),
    T_SENTENCES_OCCUR
)

df_unique = (
    spark.table(T_SENTENCES_OCCUR)
         .select("sentence_id", "sentence", "sentence_norm")
         .dropDuplicates(["sentence_norm"])
)
save_table_and_path(df_unique, T_SENTENCES_UNIQUE)

print("[INFO] sentences_occur:", spark.table(T_SENTENCES_OCCUR).count())
print("[INFO] sentences_unique:", spark.table(T_SENTENCES_UNIQUE).count())


## Step 2 - Candidate Sentence Prefilter (Top-K)
Selects a small, high-signal subset of review sentences per property for downstream semantic matching and API processing.


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Params
K_HI = 25
K_FALLBACK = 15
TARGET_MAX = K_HI + K_FALLBACK

# Load occurrences
df_occ = spark.table(T_SENTENCES_OCCUR).select(
    PROPERTY_COL, SELLER_ID_COL, "sentence_id", "sentence", "sentence_norm", "n_words"
)

# High-signal heuristic 
HIGH_SIGNAL_TERMS = [
    # negation / mismatch
    "not", "no", "never", "without", "can't", "cannot", "didn't", "doesn't", "wasn't", "weren't",
    "as described", "not as", "different than", "nothing like", "misleading", "false advertising",

    # complaints / dissatisfaction
    "complaint", "complain", "disappointed", "disappointing", "unhappy", "regret", "avoid",
    "terrible", "awful", "horrible", "worst", "bad", "poor", "mediocre", "unacceptable",

    # service / communication
    "rude", "unresponsive", "ignored", "no response", "didn't respond", "host didn't",
    "late", "delay", "delayed", "cancelled", "canceled",

    # functionality / maintenance
    "broken", "not working", "doesn't work", "didn't work", "failed", "issue", "problem",
    "malfunction", "leak", "leaking", "clogged", "blocked", "stuck", "damaged",

    # cleanliness / hygiene
    "dirty", "filthy", "unclean", "smell", "smelly", "odor", "stains", "mold", "mildew",
    "bugs", "cockroach", "roaches", "insects", "ants", "bedbug", "bed bugs", "hair",

    # noise / safety
    "noisy", "loud", "noise", "unsafe", "dangerous", "scary",

    # value / pricing
    "overpriced", "not worth", "waste", "ripoff", "scam", "expensive",

    # comfort / sleep / climate
    "uncomfortable", "hard bed", "bed hard", "small", "tiny", "cramped",
    "cold", "freezing", "hot", "heat", "ac", "air conditioning", "air conditioner", "no ac",
    "heater", "no heat", "no hot water", "hot water",
]

def _mk_or_regex(terms):
    import re
    return "(" + "|".join(re.escape(t) for t in terms) + ")"

df = (
    df_occ
    .withColumn("sent_lc", F.col("sentence_norm"))
    .withColumn("is_high_signal", F.col("sent_lc").rlike(_mk_or_regex(HIGH_SIGNAL_TERMS)))
)

# pick HI per property
w_hi = Window.partitionBy(PROPERTY_COL).orderBy(F.desc("is_high_signal"), F.desc("n_words"))
pick_hi = (
    df.filter(F.col("is_high_signal") == True)
      .withColumn("rk", F.row_number().over(w_hi))
      .filter(F.col("rk") <= F.lit(int(K_HI)))
      .drop("rk")
)

# fallback: fill remaining budget by n_words
seed_keys = pick_hi.select(PROPERTY_COL, "sentence_norm").dropDuplicates()
seed_counts = pick_hi.groupBy(PROPERTY_COL).agg(F.count("*").alias("seed_n"))

df_fb = (
    df.join(seed_counts, on=PROPERTY_COL, how="left").fillna({"seed_n": 0})
      .join(seed_keys, on=[PROPERTY_COL, "sentence_norm"], how="left_anti")
      .withColumn(
          "need_n",
          F.when(F.col("seed_n") >= F.lit(TARGET_MAX), F.lit(0))
           .otherwise(F.lit(TARGET_MAX) - F.col("seed_n"))
      )
      .withColumn("need_n", F.least(F.col("need_n"), F.lit(int(K_FALLBACK))))
)

w_fb = Window.partitionBy(PROPERTY_COL).orderBy(F.desc("n_words"))
pick_fb = (
    df_fb.withColumn("rk", F.row_number().over(w_fb))
         .filter(F.col("rk") <= F.col("need_n"))
         .drop("rk", "need_n", "seed_n")
)

# Final candidates + save
df_cand = pick_hi.unionByName(pick_fb).dropDuplicates([PROPERTY_COL, "sentence_norm"])
save_table_and_path(df_cand, T_SENTENCES_CANDIDATES_OCCUR)

df_cand_unique = (
    df_cand.select("sentence_id", "sentence", "sentence_norm")
           .dropDuplicates(["sentence_norm"])
)
save_table_and_path(df_cand_unique, T_SENTENCES_CANDIDATES_UNIQUE)

print("[INFO] prefilter in:", df_occ.count(), "| out:", df_cand.count())
print("[INFO] prefilter unique:", df_cand_unique.count())


## Step 3 – Semantic Similarity-Based Issue Matching
We compute Top1/Top2 + gap

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType

sim_schema = StructType([
    StructField("sentence_id", StringType(), True),
    StructField("sentence", StringType(), True),
    StructField("sentence_norm", StringType(), True),

    StructField("top1_issue", StringType(), True),
    StructField("top1_sim", FloatType(), True),
    StructField("top2_issue", StringType(), True),
    StructField("top2_sim", FloatType(), True),
    StructField("top_gap", FloatType(), True),

    StructField("sim_model", StringType(), True),
])

def make_similarity_once_fn_fast(model_name: str, issue_texts: dict, batch_size: int = 256, issue_batch: int = 64):
    def _fn(pdf_iter):
        import numpy as np
        import torch
        from functools import lru_cache

        # Prevent CPU thread explosion per executor
        torch.set_num_threads(1)

        @lru_cache(maxsize=1)
        def get_model_and_issue_emb():
            from sentence_transformers import SentenceTransformer
            # force cpu (since you don't have cuda currently)
            model = SentenceTransformer(model_name, device="cpu")
            issues = list(issue_texts.keys())
            issue_desc = [issue_texts[k] for k in issues]
            issue_emb = model.encode(
                issue_desc,
                normalize_embeddings=True,
                batch_size=issue_batch,
                show_progress_bar=False,
            )
            return model, issues, np.asarray(issue_emb, dtype=np.float32)

        model, issues, issue_emb = get_model_and_issue_emb()

        for pdf in pdf_iter:
            sents = pdf["sentence"].fillna("").astype(str).tolist()

            emb = model.encode(
                sents,
                normalize_embeddings=True,
                batch_size=batch_size,
                show_progress_bar=False,
            )
            emb = np.asarray(emb, dtype=np.float32)

            sims = emb @ issue_emb.T  # cosine similarity because normalized

            # Fast top-2 (no full sort)
            top2 = np.argpartition(-sims, kth=1, axis=1)[:, :2]
            row = np.arange(sims.shape[0])
            a = top2[:, 0]
            b = top2[:, 1]
            a_better = sims[row, a] >= sims[row, b]
            top1_i = np.where(a_better, a, b)
            top2_i = np.where(a_better, b, a)

            top1_sim = sims[row, top1_i].astype(np.float32)
            top2_sim = sims[row, top2_i].astype(np.float32)
            top_gap = (top1_sim - top2_sim).astype(np.float32)

            out = pdf[["sentence_id", "sentence", "sentence_norm"]].copy()
            out["top1_issue"] = [issues[i] for i in top1_i]
            out["top1_sim"] = top1_sim
            out["top2_issue"] = [issues[i] for i in top2_i]
            out["top2_sim"] = top2_sim
            out["top_gap"] = top_gap
            out["sim_model"] = "minilm"
            yield out
    return _fn

df_unique = spark.table(T_SENTENCES_CANDIDATES_UNIQUE)

# choose partitions sanely for CPU
N = max(32, min(128, spark.sparkContext.defaultParallelism * 2))
print("[INFO] Similarity partitions:", N)

fn_sim_once = make_similarity_once_fn_fast(SIM_MODEL_NAME, ISSUE_TEXTS, batch_size=128)

df_sim = (
    df_unique
    .repartition(N, "sentence_id")
    .mapInPandas(fn_sim_once, schema=sim_schema)
)

save_table_and_path(df_sim, T_SIM_UNIQUE)
print("[INFO] sim rows:", spark.table(T_SIM_UNIQUE).count())

display(spark.table(T_SIM_UNIQUE)
       .select("top1_sim","top2_sim","top_gap")
       .sample(0.001, seed=42)
       .summary("min","50%","mean","max"))

## Apply similarity thresholds (Top1 + optional Top2) BEFORE sentiment
We define candidate issue tags from similarity only.

**Rules:**
- Keep Top1 if `top1_sim >= ISSUE_SIM_THRESHOLD`
- Add Top2 only if `top2_sim >= ISSUE_SIM_THRESHOLD` AND `top_gap <= TOP2_GAP_MAX`
- `TOP_GAP_MIN` is QA signal (we keep it for analysis, not for hard drop)

In [0]:
df_sim = spark.table(T_SIM_UNIQUE)

# Top1 candidates
df_top1 = (
    df_sim
    .filter(F.col("top1_sim") >= F.lit(float(ISSUE_SIM_THRESHOLD)))
    .select(
        "sentence_id","sentence","sentence_norm",
        "top1_issue","top1_sim","top2_issue","top2_sim","top_gap","sim_model",
        F.col("top1_issue").alias("issue"),
        F.col("top1_sim").alias("issue_sim"),
        F.lit(False).alias("is_top2")
    )
)

# Top2 candidates (only when very close)
df_top2 = (
    df_sim
    .filter((F.col("top2_sim") >= F.lit(float(ISSUE_SIM_THRESHOLD))) & (F.col("top_gap") <= F.lit(float(TOP2_GAP_MAX))))
    .select(
        "sentence_id","sentence","sentence_norm",
        "top1_issue","top1_sim","top2_issue","top2_sim","top_gap","sim_model",
        F.col("top2_issue").alias("issue"),
        F.col("top2_sim").alias("issue_sim"),
        F.lit(True).alias("is_top2")
    )
)

df_issues_candidates = df_top1.unionByName(df_top2)

save_table_and_path(df_issues_candidates, T_ISSUES_CANDIDATES_UNIQUE)
print("[INFO] issue-candidate rows:", spark.table(T_ISSUES_CANDIDATES_UNIQUE).count())

icu = spark.table(T_ISSUES_CANDIDATES_UNIQUE).cache()
display(icu.groupBy("issue").count().orderBy(F.desc("count")).limit(30))


## Step 4 - Sentiment (SST-2) ONLY on the similarity-candidates
Keep where `neg_prob >= 0.40`.

In [0]:
from pyspark.sql import functions as F

N_SENT_SHARDS = 40
T_SENT_INPUT = f"airbnb_sent_input_{RUN_ID}"
T_SENT_PROGRESS = f"airbnb_sent_progress_{RUN_ID}"

cand_issues = spark.table(T_ISSUES_CANDIDATES_UNIQUE).select("sentence_id","sentence").dropDuplicates(["sentence_id"])
sent_input = (
    cand_issues
    .withColumn("sent_shard", F.pmod(F.hash(F.col("sentence_id")), F.lit(int(N_SENT_SHARDS))))
)

sent_input.write.mode("overwrite").format("delta").saveAsTable(T_SENT_INPUT)
print("[OK] Saved:", T_SENT_INPUT, "| rows:", spark.table(T_SENT_INPUT).count())

# progress table (append-only)
progress_schema = "sent_shard INT, ts STRING, rows_scored LONG"
if not spark.catalog.tableExists(T_SENT_PROGRESS):
    spark.createDataFrame([], progress_schema).write.mode("overwrite").format("delta").saveAsTable(T_SENT_PROGRESS)
    print("[OK] Init progress table:", T_SENT_PROGRESS)
else:
    print("[OK] Progress exists (resume):", T_SENT_PROGRESS, "| rows:", spark.table(T_SENT_PROGRESS).count())
print("[OK] Init progress table:", T_SENT_PROGRESS)

display(spark.table(T_SENT_INPUT).groupBy("sent_shard").count().orderBy(F.desc("count")).limit(10))

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import pandas as pd
from datetime import datetime

sent_schema = StructType([
    StructField("sentence_id", StringType(), True),
    StructField("neg_prob", FloatType(), True),
    StructField("pos_prob", FloatType(), True),
    StructField("sentiment_label", StringType(), True),
    StructField("sent_model", StringType(), True),
])

progress_schema = "sent_shard INT, ts STRING, rows_scored LONG"

def make_sst2_fn(
    model_name: str,
    batch_size_cpu: int = 64,
    batch_size_gpu: int = 256,
    max_len: int = 128,
):
    def _fn(pdf_iter):
        import os
        import numpy as np
        import torch
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        from functools import lru_cache

        # reduce overhead / thread explosion
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
        torch.set_num_threads(1)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        bs = batch_size_gpu if device.type == "cuda" else batch_size_cpu

        @lru_cache(maxsize=1)
        def get_tok_model():
            tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            mdl = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
            mdl.eval()
            return tok, mdl

        tok, mdl = get_tok_model()
        labels = ["negative", "positive"]

        use_amp = (device.type == "cuda")
        autocast_ctx = torch.cuda.amp.autocast if use_amp else None

        for pdf in pdf_iter:
            if pdf is None or pdf.empty:
                yield pd.DataFrame(columns=["sentence_id","neg_prob","pos_prob","sentiment_label","sent_model"])
                continue

            texts = pdf["sentence"].fillna("").astype(str).tolist()
            ids   = pdf["sentence_id"].astype(str).tolist()
            n = len(texts)

            neg = np.zeros(n, dtype=np.float32)
            pos = np.zeros(n, dtype=np.float32)
            lab = np.empty(n, dtype=object)

            # faster than no_grad for inference
            with torch.inference_mode():
                for start in range(0, n, bs):
                    end = min(n, start + bs)
                    enc = tok(
                        texts[start:end],
                        padding=True,
                        truncation=True,
                        max_length=max_len,
                        return_tensors="pt"
                    )
                    enc = {k: v.to(device) for k, v in enc.items()}

                    if use_amp:
                        with autocast_ctx():
                            logits = mdl(**enc).logits
                    else:
                        logits = mdl(**enc).logits

                    probs = torch.softmax(logits, dim=-1).float().cpu().numpy()
                    neg[start:end] = probs[:, 0]
                    pos[start:end] = probs[:, 1]
                    lab[start:end] = [labels[i] for i in probs.argmax(axis=1)]

            yield pd.DataFrame({
                "sentence_id": ids,
                "neg_prob": neg,
                "pos_prob": pos,
                "sentiment_label": lab,
                "sent_model": ["sst2"] * n
            })
    return _fn


# Init output/progress only if missing (resume-safe)
if not spark.catalog.tableExists(T_SENTIMENT_UNIQUE):
    spark.createDataFrame([], sent_schema).write.mode("overwrite").format("delta").saveAsTable(T_SENTIMENT_UNIQUE)
    print("[OK] Init output:", T_SENTIMENT_UNIQUE)
else:
    print("[OK] Output exists (resume):", T_SENTIMENT_UNIQUE)

if not spark.catalog.tableExists(T_SENT_PROGRESS):
    spark.createDataFrame([], progress_schema).write.mode("overwrite").format("delta").saveAsTable(T_SENT_PROGRESS)
    print("[OK] Init progress:", T_SENT_PROGRESS)
else:
    print("[OK] Progress exists (resume):", T_SENT_PROGRESS)


fn_sst2 = make_sst2_fn(SENT_MODEL_NAME)

# Partitions: keep small to reduce overhead per shard 
P = max(1, min(8, spark.sparkContext.defaultParallelism * 2))
print("[INFO] P:", P, "| defaultParallelism:", spark.sparkContext.defaultParallelism)

# Done shards
done = set(r["sent_shard"] for r in spark.table(T_SENT_PROGRESS).select("sent_shard").distinct().collect())
print(f"[INFO] already done shards: {len(done)} / {N_SENT_SHARDS}")

# Optional health: show biggest shards
print("[QA] Largest sentiment shards (top 10)")
display(
    spark.table(T_SENT_INPUT)
         .groupBy("sent_shard")
         .count()
         .orderBy(F.desc("count"))
         .limit(10)
)

# Main loop
for k in range(N_SENT_SHARDS):
    if k in done:
        continue

    df_k = (
        spark.table(T_SENT_INPUT)
             .filter(F.col("sent_shard") == F.lit(int(k)))
             .select("sentence_id","sentence")
    )

    n_in = df_k.count()
    print(f"\n[RUN] sent_shard={k} | input rows={n_in:,}")
    if n_in == 0:
        spark.createDataFrame([(k, datetime.utcnow().isoformat(), 0)], progress_schema) \
            .write.mode("append").format("delta").saveAsTable(T_SENT_PROGRESS)
        continue

    scored = (
        df_k.coalesce(P)
            .mapInPandas(fn_sst2, schema=sent_schema)
    )

    scored.write.mode("append").format("delta").saveAsTable(T_SENTIMENT_UNIQUE)

    spark.createDataFrame([(k, datetime.utcnow().isoformat(), n_in)], progress_schema) \
         .write.mode("append").format("delta").saveAsTable(T_SENT_PROGRESS)

    # light prints: do NOT count the whole output table each shard
    if (k % 10) == 0:
        done_now = spark.table(T_SENT_PROGRESS).select("sent_shard").distinct().count()
        out_rows = spark.table(T_SENT_PROGRESS).agg(F.sum("rows_scored").alias("rows")).collect()[0]["rows"]
        print(f"[OK] shard {k} appended | approx_rows_scored={out_rows:,} | shards_done={done_now}/{N_SENT_SHARDS}")
    else:
        print(f"[OK] shard {k} appended | shards_done=? (see every 10 shards)")


### Build NEG_UNIQUE (neg_prob >= threshold)

In [0]:
from pyspark.sql import functions as F

sent = spark.table(T_SENTIMENT_UNIQUE)

neg_unique = (
    sent.filter(F.col("neg_prob") >= F.lit(float(NEG_PROB_THRESHOLD)))
        .select("sentence_id","neg_prob","pos_prob","sentiment_label","sent_model")
        .dropDuplicates(["sentence_id"])
)

save_table_and_path(neg_unique, T_NEG_UNIQUE)
print("[INFO] neg_unique rows:", spark.table(T_NEG_UNIQUE).count())
display(neg_unique.select("neg_prob").summary("min","25%","50%","mean","75%","max"))


## Step 5 - Join: (Issue candidates) × (Negative sentiment) → final issues per sentence
Then map back to occurrences to preserve property/host linkage.

In [0]:
issues = spark.table(T_ISSUES_CANDIDATES_UNIQUE)
neg = spark.table(T_NEG_UNIQUE)

df_final_issues_unique = (
    issues.join(neg.select("sentence_id","neg_prob","pos_prob","sentiment_label"), on="sentence_id", how="inner")
)

save_table_and_path(df_final_issues_unique, T_ISSUES_UNIQUE)
iu2 = spark.table(T_ISSUES_UNIQUE)
print("[OK] Final issues unique saved:", T_ISSUES_UNIQUE, "| rows:", iu2.count())

print("[OK] Final issues unique saved (overwrites candidates):", T_ISSUES_UNIQUE, "| rows:", iu2.count())
display(iu2.groupBy("issue").count().orderBy(F.desc("count")).limit(30))
display(iu2.orderBy(F.desc("neg_prob")).select("neg_prob","issue","issue_sim","sentence").limit(30))

# Map to occurrences
occ = spark.table(T_SENTENCES_CANDIDATES_OCCUR)

df_issues_occur = occ.join(iu2.select("sentence_id","issue","issue_sim","is_top2","top_gap","top1_issue","top2_issue","top1_sim","top2_sim","neg_prob"), on="sentence_id", how="inner")

save_table_and_path(df_issues_occur, T_ISSUES_OCCUR)

io = spark.table(T_ISSUES_OCCUR).cache()
print("[OK] Saved:", T_ISSUES_OCCUR, "| rows:", io.count())
display(io.groupBy("issue").count().orderBy(F.desc("count")).limit(30))
display(io.orderBy(F.desc("issue_sim")).select(PROPERTY_COL, SELLER_ID_COL, "issue", "issue_sim", "neg_prob", "sentence").limit(30))

## Step 6 - Aggregations (Property + Host) + Evidence tables

In [0]:
df = spark.table(T_ISSUES_OCCUR)

# Property-level aggregation
prop_agg = (
    df.groupBy(PROPERTY_COL, SELLER_ID_COL, "issue")
      .agg(
          F.count("*").alias("mentions"),
          F.avg("issue_sim").alias("avg_issue_sim"),
          F.stddev("issue_sim").alias("std_issue_sim"),
          F.expr("percentile_approx(issue_sim, 0.5)").alias("median_issue_sim"),
          F.avg("neg_prob").alias("avg_neg_prob"),
          F.stddev("neg_prob").alias("std_neg_prob"),
          F.expr("percentile_approx(neg_prob, 0.5)").alias("median_neg_prob"),
          F.max("neg_prob").alias("max_neg_prob"),
      )
      .withColumn("avg_issue_sim", F.round("avg_issue_sim", 3))
      .withColumn("std_issue_sim", F.round("std_issue_sim", 3))
      .withColumn("median_issue_sim", F.round("median_issue_sim", 3))
      .withColumn("avg_neg_prob", F.round("avg_neg_prob", 3))
      .withColumn("std_neg_prob", F.round("std_neg_prob", 3))
      .withColumn("median_neg_prob", F.round("median_neg_prob", 3))
      .withColumn("max_neg_prob", F.round("max_neg_prob", 3))
)

save_table_and_path(prop_agg, T_PROP_ISSUE_AGG)
print("[OK] Saved:", T_PROP_ISSUE_AGG, "| rows:", spark.table(T_PROP_ISSUE_AGG).count())
display(spark.table(T_PROP_ISSUE_AGG).limit(50))

# Host-level aggregation
host_agg = (
    df.groupBy(SELLER_ID_COL, "issue")
      .agg(
          F.count("*").alias("mentions"),
          F.countDistinct(PROPERTY_COL).alias("n_properties"),
          F.avg("issue_sim").alias("avg_issue_sim"),
          F.expr("percentile_approx(issue_sim, 0.5)").alias("median_issue_sim"),
          F.avg("neg_prob").alias("avg_neg_prob"),
      )
      .withColumn("avg_issue_sim", F.round("avg_issue_sim", 3))
      .withColumn("median_issue_sim", F.round("median_issue_sim", 3))
      .withColumn("avg_neg_prob", F.round("avg_neg_prob", 3))
      .orderBy(F.desc("mentions"))
)

save_table_and_path(host_agg, T_HOST_ISSUE_AGG)
print("[OK] Saved:", T_HOST_ISSUE_AGG, "| rows:", spark.table(T_HOST_ISSUE_AGG).count())
display(spark.table(T_HOST_ISSUE_AGG).limit(50))

# Evidence tables
K_EVID = 5

w_prop = Window.partitionBy(PROPERTY_COL, SELLER_ID_COL, "issue") \
               .orderBy(F.desc("issue_sim"), F.desc("neg_prob"))

prop_evid = (df.withColumn("ev_rank", F.row_number().over(w_prop))
      .select(PROPERTY_COL, SELLER_ID_COL, "issue", "ev_rank",
          "issue_sim", "neg_prob", "sentence", "sentence_id",
          "is_top2","top_gap","top1_issue","top2_issue"))

save_table_and_path(prop_evid, T_PROP_ISSUE_EVID)
print("[OK] Saved:", T_PROP_ISSUE_EVID, "| rows:", spark.table(T_PROP_ISSUE_EVID).count())
display(spark.table(T_PROP_ISSUE_EVID).limit(50))

w_host = Window.partitionBy(SELLER_ID_COL, "issue") \
               .orderBy(F.desc("issue_sim"), F.desc("neg_prob"))

host_evid = (df.withColumn("ev_rank", F.row_number().over(w_host))
      .select(SELLER_ID_COL, "issue", "ev_rank",
          PROPERTY_COL, "issue_sim", "neg_prob",
          "sentence", "sentence_id",
          "is_top2","top_gap","top1_issue","top2_issue"))

save_table_and_path(host_evid, T_HOST_ISSUE_EVID)
print("[OK] Saved:", T_HOST_ISSUE_EVID, "| rows:", spark.table(T_HOST_ISSUE_EVID).count())
display(spark.table(T_HOST_ISSUE_EVID).limit(50))


## Step 7 - Global summaries (Top issues, most problematic properties/hosts)

In [0]:
df_prop = spark.table(T_PROP_ISSUE_AGG)
df_host = spark.table(T_HOST_ISSUE_AGG)

print("[REPORT] Global top issues (by mentions)")
global_issues = (
    df_prop.groupBy("issue")
           .agg(
               F.sum("mentions").alias("mentions"),
               F.countDistinct(PROPERTY_COL).alias("n_properties"),
               F.avg("avg_neg_prob").alias("avg_neg_prob"),
               F.avg("avg_issue_sim").alias("avg_issue_sim"),
           )
           .withColumn("avg_neg_prob", F.round("avg_neg_prob", 3))
           .withColumn("avg_issue_sim", F.round("avg_issue_sim", 3))
           .orderBy(F.desc("mentions"))
)
display(global_issues.limit(30))

print("[REPORT] Most problematic properties (high mentions + high neg_prob)")
worst_props = (
    df_prop.withColumn("red_flag", F.col("mentions") * F.col("avg_neg_prob") * F.col("avg_issue_sim"))
           .orderBy(F.desc("red_flag"))
)
display(worst_props.limit(50))

print("[REPORT] Most problematic hosts (high mentions across many properties)")
worst_hosts = (
    df_host.withColumn("red_flag", F.col("mentions") * F.col("avg_neg_prob") * F.col("avg_issue_sim"))
           .orderBy(F.desc("red_flag"))
)
display(worst_hosts.limit(50))

example = worst_props.select(PROPERTY_COL, SELLER_ID_COL).limit(1).collect()
if example:
    pid = example[0][PROPERTY_COL]
    sid = example[0][SELLER_ID_COL]
    print(f"[REPORT] Example property drilldown: property_id={pid} | seller_id={sid}")

    display(
        spark.table(T_PROP_ISSUE_AGG)
             .filter((F.col(PROPERTY_COL) == pid) & (F.col(SELLER_ID_COL) == sid))
             .orderBy(F.desc("mentions"), F.desc("avg_neg_prob"))
    )
    display(
        spark.table(T_PROP_ISSUE_EVID)
             .filter((F.col(PROPERTY_COL) == pid) & (F.col(SELLER_ID_COL) == sid))
             .orderBy("issue","ev_rank")
    )

print("[OK] Pipeline complete. All tables saved with RUN_ID:", RUN_ID)

# LLM recommendations

In [0]:
# =============================
# GOOGLE API KEY (REQUIRED)
# =============================
# Insert your personal Google Generative AI API key below.
# This key is used only to generate live LLM recommendations
# and is NOT stored or shared.

GOOGLE_API_KEY = ""   # <-- paste your API key here

In [0]:
# API config

# GOOGLE_API_KEY = None
GEMINI_MODEL = "models/gemini-2.5-pro"
API_SLEEP_SEC = 0.25
API_TIMEOUT_SEC = 120

# Output tables
T_API_TARGETS = f"airbnb_api_targets_{RUN_ID}"
T_API_RESULTS = f"airbnb_api_recs_{RUN_ID}"

api_schema = "property_id STRING, seller_id STRING, ts STRING, request_json STRING, response_json STRING, status STRING, error STRING"

if GOOGLE_API_KEY is None:
    print("[WARN] GOOGLE_API_KEY is not set. API calls will fail until you load it from secrets/env.")
print("[INFO] API targets table:", T_API_TARGETS)
print("[INFO] API results table:", T_API_RESULTS)


In [0]:
from pyspark.sql import functions as F

PROPERTY_IDS = [
    "1042005770541410920",
    "10566180",
    "11251348",
    "1166472471627354698",
    "1259994760695336460",
    "14410929",
    "1773055",
]

# Evidence table (property-level)
EVID_TABLE = T_PROP_ISSUE_EVID

# Filter evidence to the chosen properties
evid = (
    spark.table(EVID_TABLE)
         .filter(F.col(PROPERTY_COL).cast("string").isin(PROPERTY_IDS))
)

# Keep only Top1 evidence (if table contains both top1/top2 info)
# If your evidence table has an "is_top2" flag, this keeps only top1 rows.
if "is_top2" in evid.columns:
    evid = evid.filter((F.col("is_top2").isNull()) | (F.col("is_top2") == F.lit(False)))

# Build targets (one row per property+seller)
picked = (
    evid.groupBy(PROPERTY_COL, SELLER_ID_COL)
        .agg(F.count("*").alias("survived_sentences"))
        .orderBy(F.desc("survived_sentences"))
)

save_table_and_path(picked, T_API_TARGETS)

display(picked)
print("[OK] Targets saved:", T_API_TARGETS)


In [0]:
import json
from pyspark.sql import functions as F

def build_payload_for_property(property_id: str, seller_id: str, max_sentences: int = 120):
    """
    Build a compact payload for the LLM (one property):
    - Input: negative + issue-tagged sentences for (property_id, seller_id)
    - Output: a compact dict with metadata + sentences list
    """
    df = (spark.table(T_ISSUES_OCCUR)
             .filter((F.col(PROPERTY_COL) == str(property_id)) & (F.col(SELLER_ID_COL) == str(seller_id)))
             .orderBy(F.desc("neg_prob"), F.desc("issue_sim"))
             .limit(int(max_sentences))
             .select(
                 PROPERTY_COL, SELLER_ID_COL,
                 "sentence_id", "sentence",
                 "issue", "issue_sim",
                 "neg_prob",
                 "is_top2", "top_gap", "top1_issue", "top2_issue"))

    rows = [r.asDict(True) for r in df.collect()]

    return {
        "entity_type": "property",
        "property_id": str(property_id),
        "seller_id": str(seller_id),
        "sentences": rows
    }

In [0]:
import time, requests
from datetime import datetime

def call_gemini_api(payload: dict, timeout_sec: int = 120):
    if not GOOGLE_API_KEY:
        return False, 0, None, "GOOGLE_API_KEY is not set"

    url = f"https://generativelanguage.googleapis.com/v1beta/{GEMINI_MODEL}:generateContent?key={GOOGLE_API_KEY}"

    prompt_text = (
        "You are an expert hospitality operations consultant.\n\n"
        "Analyze the following Airbnb feedback for ONE property.\n"
        "Each item includes: sentence, neg_prob (probability the sentence is negative), and issue category.\n\n"
        "Return a short summary and practical recommendations for the host.\n\n"
        "OUTPUT FORMAT (strict):\n"
        "Summary: <2-3 concise sentences describing the main problems and their impact>\n"
        "Priority: P0 or P1 (P0 = urgent/critical, P1 = important but not critical)\n"
        "Practical recommendations:\n"
        "- <actionable recommendation 1>\n"
        "- <actionable recommendation 2>\n"
        "- <actionable recommendation 3>\n"
        "(2-6 bullets total; be specific and implementable, e.g., noise -> door seals / double glazing / thick curtains)\n\n"
        "Do NOT return JSON. Do NOT add extra sections or titles. Do NOT cite sentence_ids.\n\n"
        f"DATA:\n{json.dumps(payload, ensure_ascii=False, indent=2)}"
    )

    body = {"contents": [{"role": "user", "parts": [{"text": prompt_text}]}]}
    r = requests.post(url, json=body, timeout=timeout_sec)

    if 200 <= r.status_code < 300:
        return True, r.status_code, r.json(), None
    return False, r.status_code, None, (r.text[:2000] if r.text else "HTTP error")

# Ensure results table exists
if not spark.catalog.tableExists(T_API_RESULTS):
    spark.createDataFrame([], api_schema).write.mode("overwrite").format("delta").saveAsTable(T_API_RESULTS)
    print("[OK] Init API results table:", T_API_RESULTS)

targets = spark.table(T_API_TARGETS).collect()
print("[INFO] API targets:", [(t[PROPERTY_COL], t[SELLER_ID_COL], t["survived_sentences"]) for t in targets])

rows_to_save = []

for t in targets:
    pid = t[PROPERTY_COL]
    sid = t[SELLER_ID_COL]

    payload = build_payload_for_property(pid, sid)

    try:
        ok, code, resp_json, err = call_gemini_api(payload, timeout_sec=int(API_TIMEOUT_SEC))
        rows_to_save.append((
            str(pid), str(sid),
            datetime.utcnow().isoformat(),
            json.dumps(payload, ensure_ascii=False),
            json.dumps(resp_json, ensure_ascii=False) if resp_json is not None else None,
            "ok" if ok else f"http_{code}",
            err
        ))
        print(f"[API] property={pid} seller={sid} -> {'OK' if ok else 'ERR'} ({code})")
    except Exception as e:
        rows_to_save.append((
            str(pid), str(sid),
            datetime.utcnow().isoformat(),
            json.dumps(payload, ensure_ascii=False),
            None,
            "exception",
            repr(e)
        ))
        print(f"[API] property={pid} seller={sid} -> EXCEPTION: {e}")

    time.sleep(float(API_SLEEP_SEC))

df_out = spark.createDataFrame(rows_to_save, api_schema)

# append to metastore table
df_out.write.mode("append").format("delta").saveAsTable(T_API_RESULTS)

# also append to OUTPUT_BASE path
df_out.write.mode("append").format("delta").save(f"{OUTPUT_BASE}/{T_API_RESULTS}")

display(spark.table(T_API_RESULTS).orderBy(F.desc("ts")))
print("[OK] Saved:", T_API_RESULTS, "| appended_rows:", df_out.count())

In [0]:
from pyspark.sql import functions as F

PROPERTY_IDS = [
    "1042005770541410920",
    "10566180",
    "11251348",
    "1166472471627354698",
    "1259994760695336460",
    "14410929",
    "1773055",
]

api_ok = (
    spark.table(T_API_RESULTS)
         .filter(F.col("status") == "ok")
         .filter(F.col("property_id").cast("string").isin(PROPERTY_IDS))   # <-- במקום RUN_DATE
         .select(
             F.col("property_id").cast("string").alias("property_id"),
             F.col("seller_id").cast("string").alias("seller_id"),
             F.get_json_object(
                 "response_json",
                 "$.candidates[0].content.parts[0].text"
             ).alias("llm_text"),
             "ts"
         )
)

agg = (
    spark.table(T_PROP_ISSUE_AGG)
         .select(
             F.col(PROPERTY_COL).cast("string").alias("property_id"),
             F.col(SELLER_ID_COL).cast("string").alias("seller_id"),
             "issue",
             "mentions",
             "avg_issue_sim",
             "median_issue_sim",
             "avg_neg_prob",
             "median_neg_prob",
             "max_neg_prob"
         )
)

api_prop_issue = (
    api_ok
    .join(agg, on=["property_id", "seller_id"], how="inner")
)

display(api_prop_issue.orderBy(F.desc("mentions")).limit(50))

evid = (
    spark.table(T_PROP_ISSUE_EVID)
         .select(
             F.col(PROPERTY_COL).cast("string").alias("property_id"),
             F.col(SELLER_ID_COL).cast("string").alias("seller_id"),
             "issue",
             "sentence_id",
             "sentence",
             "neg_prob",
             "issue_sim"
         )
)

api_prop_issue_evid = (
    api_prop_issue
    .join(evid, on=["property_id","seller_id","issue"], how="left")
)

display(api_prop_issue_evid.limit(50))

T_API_PROP_ISSUE = "airbnb_api_property_issue_ITALY_1"
T_API_PROP_ISSUE_EVID = "airbnb_api_property_issue_evidence_ITALY_1"

save_table_and_path(api_prop_issue, T_API_PROP_ISSUE)
save_table_and_path(api_prop_issue_evid, T_API_PROP_ISSUE_EVID)

In [0]:
from pyspark.sql import functions as F

# Property+Issue aggregate table (mentions + stats)
agg_all = (
    spark.table(T_PROP_ISSUE_AGG)
         .select(
             F.col(PROPERTY_COL).cast("string").alias("property_id"),
             F.col(SELLER_ID_COL).cast("string").alias("seller_id"),
             "issue",
             "mentions",
             "avg_issue_sim",
             "median_issue_sim",
             "avg_neg_prob",
             "median_neg_prob",
             "max_neg_prob"
         )
)

# Evidence table (all sentences per property+issue)
evid_all = (
    spark.table(T_PROP_ISSUE_EVID)
         .select(
             F.col(PROPERTY_COL).cast("string").alias("property_id"),
             F.col(SELLER_ID_COL).cast("string").alias("seller_id"),
             "issue",
             "sentence_id",
             "sentence",
             "neg_prob",
             "issue_sim",
             *([c for c in ["ev_rank", "sent_shard", "ts"] if c in spark.table(T_PROP_ISSUE_EVID).columns])
         )
)

# Full normalized table: one row per sentence evidence, with agg stats duplicated
prop_issue_sentence_full = (
    agg_all
    .join(evid_all, on=["property_id", "seller_id", "issue"], how="left")
)

display(prop_issue_sentence_full.orderBy(F.desc("mentions")).limit(50))

# Save
T_PROP_ISSUE_SENT_FULL = f"airbnb_property_issue_sentence_full_{RUN_ID}"
save_table_and_path(prop_issue_sentence_full, T_PROP_ISSUE_SENT_FULL)
print("Saved:", T_PROP_ISSUE_SENT_FULL)
